In [1]:
!nvidia-smi

Tue May  3 15:35:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   54C    P0    37W / 250W |   7701MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [2]:
# !pip install git+https://github.com/OpenNMT/OpenNMT-py.git
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 552 kB/s eta 0:00:01
     |████████████████████████████████| 73 kB 516 kB/s eta 0:00:011
     |████████████████████████████████| 95 kB 3.9 MB/s eta 0:00:011
     |████████████████████████████████| 16.6 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 791 kB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 224 kB 840 kB/s eta 0:00:01
     |████████████████████████████████| 96 kB 947 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 882 kB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 952 kB/s eta 0:00:01
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.6.0
    Uninstalling torchtext-0.6.0:
      Successfully uninstalled torchtext-0.6.0
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0

In [3]:
!git clone https://bsantraigi:@github.com/bsantraigi/2022-R3-Mirror-Baseline

Cloning into '2022-R3-Mirror-Baseline'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 118 (delta 38), reused 109 (delta 29), pack-reused 0
Receiving objects: 100% (118/118), 15.10 MiB | 8.11 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [4]:
%cd 2022-R3-Mirror-Baseline/

/home/bishal/HULK/Projects/2022-r3-opennmt-dialog/2022-R3-Mirror-Baseline


In [5]:
%%bash
wget -q -c http://yanran.li/files/ijcnlp_dailydialog.zip
unzip ijcnlp_dailydialog.zip -d data/
rm ijcnlp_dailydialog.zip

unzip data/ijcnlp_dailydialog/test.zip -d data/ijcnlp_dailydialog/
unzip data/ijcnlp_dailydialog/train.zip -d data/ijcnlp_dailydialog/
unzip data/ijcnlp_dailydialog/validation.zip -d data/ijcnlp_dailydialog/

python preprocess_dd.py
# python -u preprocess.py \
#   -train_src data/ijcnlp_dailydialog/train.src.txt \
#   -train_tgt data/ijcnlp_dailydialog/train.tgt.txt \
#   -train_ctx data/ijcnlp_dailydialog/train.ctx.txt \
#   -valid_src data/ijcnlp_dailydialog/validation.src.txt \
#   -valid_tgt data/ijcnlp_dailydialog/validation.tgt.txt \
#   -valid_ctx data/ijcnlp_dailydialog/validation.ctx.txt \
#   -save_data data/ijcnlp_dailydialog/pair_daily -dynamic_dict -share_vocab \
#   -src_seq_length 45 -ctx_seq_length 100 -tgt_seq_length 45

Archive:  ijcnlp_dailydialog.zip
   creating: data/ijcnlp_dailydialog/
  inflating: data/ijcnlp_dailydialog/.DS_Store  
  inflating: data/ijcnlp_dailydialog/dialogues_act.txt  
  inflating: data/ijcnlp_dailydialog/dialogues_emotion.txt  
  inflating: data/ijcnlp_dailydialog/dialogues_text.txt  
  inflating: data/ijcnlp_dailydialog/dialogues_topic.txt  
  inflating: data/ijcnlp_dailydialog/readme.txt  
  inflating: data/ijcnlp_dailydialog/test.zip  
 extracting: data/ijcnlp_dailydialog/train.zip  
  inflating: data/ijcnlp_dailydialog/validation.zip  
Archive:  data/ijcnlp_dailydialog/test.zip
   creating: data/ijcnlp_dailydialog/test/
  inflating: data/ijcnlp_dailydialog/test/dialogues_act_test.txt  
  inflating: data/ijcnlp_dailydialog/test/dialogues_emotion_test.txt  
  inflating: data/ijcnlp_dailydialog/test/dialogues_test.txt  
Archive:  data/ijcnlp_dailydialog/train.zip
   creating: data/ijcnlp_dailydialog/train/
  inflating: data/ijcnlp_dailydialog/train/dialogues_act_train.txt  


100%|██████████| 1000/1000 [00:00<00:00, 163680.16it/s]


In [6]:
!mkdir -p configs/

In [7]:
%%writefile configs/dd_data.yaml
# toy_en_de.yaml

## Where the samples will be written
save_data: data/ijcnlp_dailydialog/run/

## Where the vocab(s) will be written
src_vocab: data/ijcnlp_dailydialog/run/vocab.src
tgt_vocab: data/ijcnlp_dailydialog/run/vocab.tgt

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: data/ijcnlp_dailydialog/train.src.txt
        path_tgt: data/ijcnlp_dailydialog/train.tgt.txt
    valid:
        path_src: data/ijcnlp_dailydialog/validation.src.txt
        path_tgt: data/ijcnlp_dailydialog/validation.tgt.txt

Writing configs/dd_data.yaml


In [8]:
!onmt_build_vocab -config configs/dd_data.yaml -n_sample 1000000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-05-02 21:19:52,766 INFO] Counter vocab from 1000000 samples.
[2022-05-02 21:19:52,766 INFO] Build vocab on 1000000 transformed examples/corpus.
[2022-05-02 21:19:52,785 INFO] corpus_1's transforms: TransformPipe()
[2022-05-02 21:19:53,591 INFO] Counters src:22431
[2022-05-02 21:19:53,591 INFO] Counters tgt:22088


In [14]:
%%writefile configs/dd_transformer_wordvocab_model.yaml
## Where the samples will be written
save_data: data/ijcnlp_dailydialog/run/

## Where the vocab(s) will be written
src_vocab: data/ijcnlp_dailydialog/run/vocab.src
tgt_vocab: data/ijcnlp_dailydialog/run/vocab.tgt

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: data/ijcnlp_dailydialog/train.src.txt
        path_tgt: data/ijcnlp_dailydialog/train.tgt.txt
    valid:
        path_src: data/ijcnlp_dailydialog/validation.src.txt
        path_tgt: data/ijcnlp_dailydialog/validation.tgt.txt

# ----------------------------
# data: data/ijcnlp_dailydialog/run/

# Vocabulary files that were just created
src_vocab: data/ijcnlp_dailydialog/run/vocab.src
tgt_vocab: data/ijcnlp_dailydialog/run/vocab.tgt

save_model: data/ijcnlp_dailydialog/run/transformer_wordvocab
save_checkpoint_steps: 10000
keep_checkpoint: 10
seed: 3435
train_steps: 500000
valid_steps: 10000
warmup_steps: 8000
report_every: 100

decoder_type: transformer
encoder_type: transformer
word_vec_size: 512
rnn_size: 512
layers: 6
transformer_ff: 2048
heads: 8

accum_count: 8
optim: adam
adam_beta1: 0.9
adam_beta2: 0.998
decay_method: noam
learning_rate: 2.0
max_grad_norm: 0.0

batch_size: 8192
batch_type: tokens
normalization: tokens
dropout: 0.1
label_smoothing: 0.1

max_generator_batches: 2

param_init: 0.0
param_init_glorot: 'true'
position_encoding: 'true'

world_size: 1
gpu_ranks:
- 0

Overwriting configs/dd_transformer_wordvocab_model.yaml


In [ ]:
!onmt_train -config configs/dd_transformer_wordvocab_model.yaml